In [ ]:
# this is ti process the vacant data and find relations between new vacant lots and their zip code. We will be using geocoding_look API to get the zip code of the vacant lots.


In [44]:
import geocoding_look as geo
import pandas as pd
import numpy as np
import time

In [6]:
# read the vacant data
vacant = pd.read_csv('Vacant_Property_Registrations.csv')

In [7]:
vacant.head()

,X,Y,record_id,street_num,street_dir,street_name,date_status,address_id,parcel_id,lon,lat,ObjectId
0,-83.087269,42.380564,VPO2011-03556,751,NaN,CLAIRMOUNT,2022/05/04 04:00:00+00,344110.0,04002447.,-83.087269,42.380564,1
1,-83.058214,42.438002,VPO2023-00230,19455,NaN,EUREKA,2023/02/21 05:00:00+00,291700.0,13018561-2,-83.058214,42.438002,2
2,-83.211644,42.443471,VPO2011-05004,20543,NaN,FERGUSON,2023/04/05 04:00:00+00,73135.0,22063524.,-83.211644,42.443471,3
3,-83.233315,42.363661,VPO2023-00232,9346,NaN,AUBURN,2023/02/15 05:00:00+00,173604.0,22092326-7,-83.233315,42.363661,4
4,-83.220578,42.344355,VPO2011-05239,7264,NaN,FAUST,2023/09/21 04:00:00+00,161210.0,22078398.,-83.220578,42.344355,5


In [29]:
vacant['street_num'] = vacant['street_num'].astype('str')

In [30]:
# generate full address
def address_gen(row):
    if pd.isnull(row['street_dir']):
        return row['street_num'] + ' ' + row['street_name'] + ", Detroit, MI"
    else:
        return row['street_num'] + ' ' + row['street_dir'] + ' ' + row['street_name'] + ", Detroit, MI"

In [31]:
vacant['full_address'] = vacant.apply(address_gen, axis=1)

In [32]:
vacant.tail()

,X,Y,record_id,street_num,street_dir,street_name,date_status,address_id,parcel_id,lon,lat,ObjectId,full_address
2966,-83.173225,42.411571,VPO2023-00223,16245,NaN,SORRENTO,2023/02/21 05:00:00+00,88411.0,22024074.,-83.173225,42.411571,2967,"16245 SORRENTO, Detroit, MI"
2967,-83.158599,42.368771,VPO2023-00224,9825,NaN,WYOMING,2023/02/13 05:00:00+00,208400.0,18016094.,-83.158599,42.368771,2968,"9825 WYOMING, Detroit, MI"
2968,-83.134711,42.310714,VPO2023-00225,2440,NaN,CABOT,2023/02/21 05:00:00+00,35631.0,20007722.,-83.134711,42.310714,2969,"2440 CABOT, Detroit, MI"
2969,-82.942516,42.402125,VPO2023-00226,5035,NaN,DEVONSHIRE,2023/02/21 05:00:00+00,320911.0,21070600.,-82.942516,42.402125,2970,"5035 DEVONSHIRE, Detroit, MI"
2970,-83.234550,42.347674,VPO2023-00229,7659,NaN,PLAINVIEW,2023/02/21 05:00:00+00,167197.0,22094777.,-83.234550,42.347674,2971,"7659 PLAINVIEW, Detroit, MI"


In [45]:
# use geocoing_look API to get the zip code
zip_codes = []
geoer = geo.geoLookup()
for address in vacant['full_address']:
    try:
        zip_codes.append(geoer.getZipCode(address))
    except:
        zip_codes.append(np.nan)
    time.sleep(0.001)

In [47]:
vacant['zip_code'] = zip_codes

In [49]:
vacant.zip_code.isna().sum()

75

In [50]:
# export the data
vacant.to_csv('vacant_with_zip.csv', index=False)